In [ ]:
# Install required packages and download data
%pip install -r requirements.txt
!sh data_get.sh


In [ ]:

!sh data_wrangle.sh

In [ ]:
import duckdb
import pandas as pd
from collections import defaultdict
from os import listdir #, getenv
# from dotenv import load_dotenv
# load_dotenv()

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# %sql duckdb:///cerulean.db
%sql duckdb:///:memory:

In [ ]:
# Define a function to create a schema for each data source
# and a table for each file (*.csv.gz) in its directory

# Specify NULLSTR for problematic tables
options = defaultdict(lambda: '')
options['mimiciv_hosp.discharge'] = ', NULLSTR="___"'
options['mimiciv_hosp.emar_detail'] = ', NULLSTR="___"'
options['mimiciv_hosp.pharmacy.csv.gz'] = ', NULLSTR="___"'

def create_schema_and_tables(schema, path):
    files = listdir(path)
    %sql CREATE SCHEMA IF NOT EXISTS {schema};
    for file in files:
        if file.endswith('.csv.gz'):
            table = file.split('.')[0]
            param = 'AUTO_DETECT=TRUE' + options[schema + '.' + table]
            %sql DROP TABLE IF EXISTS {schema}.{table};
            %sql CREATE TABLE {schema}.{table} AS SELECT * FROM read_csv('{path}{file}', {param} );
    %sql result << SELECT table_schema, table_name FROM information_schema.tables WHERE table_schema = '{schema}';
    return result

In [ ]:
# Import MIMIC-IV Hospital
schema = 'mimiciv_hosp'
path = 'physionet.org/files/mimiciv/2.2/hosp/'

create_schema_and_tables(schema, path)

In [ ]:
# Import MIMIC-IV ICU
schema = 'mimiciv_icu'
path = 'physionet.org/files/mimiciv/2.2/icu/'

create_schema_and_tables(schema, path)

In [ ]:
# Import MIMIC-IV Notes
schema = 'mimiciv_note'
path = 'physionet.org/files/mimic-iv-note/2.2/note/'

create_schema_and_tables(schema, path)